# Quickmodel
A simple command project I'm putting together. Goal: User runs the file with a specific search term, and the program returns a pickled ML model that is trained to recognize images of the search term. 

In [18]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
import matplotlib.pyplot as plt
import os
img_dir = os.listdir('../data/processed')
img_count = len(img_dir)
img_count

29

First, let's generate our dataset from our existing files. Luckily, ``keras`` has a nice way of implementing this from their Image library.

In [28]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
BATCH_SIZE = 10
IMG_HEIGHT = 256
IMG_WIDTH = 256
STEPS_PER_EPOCH = np.ceil(img_count/BATCH_SIZE)

train_data_gen = image_generator.flow_from_directory(directory='../data/',
                                                     batch_size=3,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = ['Dogs']) #should come from script parameter


Found 29 images belonging to 1 classes.


Now that we have our dataset, we need to construct a model and train it.